In [2]:
import numpy as np 
import pandas as pd 
import os
from glob import glob
from random import shuffle
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Input, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from keras.callbacks import ModelCheckpoint
import math
import random
import gc
from keras.callbacks import *
from keras.applications.vgg19 import VGG19
from keras.utils import to_categorical


from google.colab import drive
drive.mount('/content/gdrive')
#unzip the dataset
!unzip 'gdrive/My Drive/train-scene classification.zip'
#read the csv file
df_raw=pd.read_csv("train.csv")#read csv file

#function to load all images and their corresponding labels
IMAGES_PATH="train/{}"
def load_images_train(data,images_path):
    resize=(150,150)
    x=[]
    y=[]
    ids=[]
    i=0
    for img_id,lbl in data.values:
        i=i+1
        if(i%1000==0):
           print("{}/{}".format(i,len(data)))
        image=cv2.imread(images_path.format(img_id))
        x.append(cv2.resize(image,resize))
        y.append(lbl)
        ids.append(img_id)
        
    return x,y,ids

#load data
data,labels,ids=load_images_train(df_raw,IMAGES_PATH)

#now let's split the dataset into training and validation data
X_train,X_test,y_train,y_test=train_test_split(data,labels,test_size=0.30)
#prepare the dataset so that it could be feeded to model
X_train=np.stack(X_train,axis=0)
X_test=np.stack(X_test,axis=0)
y_train=to_categorical(y_train,num_classes=6)
y_test=to_categorical(y_test,num_classes=6)


#create a base model
base_model3 = VGG19(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

NUM_CLASSES=6
#freeze all layers in base_model
for layers in base_model3.layers:
    layers.trainable=False
#now stack more layers on the model
model3 = Sequential([
    base_model3,
    Flatten(),
    Dense(2048, activation='relu'),
    Dropout(rate=0.5),
    Dense(512, activation='relu'),
    Dropout(rate=0.5),
    Dense(NUM_CLASSES, activation='softmax')
])
#create an optimizer
optimizer = Adam(0.00001,decay=0.000001)
model3.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
model3.summary()

#Now let's train the model
history=model3.fit(X_train,y_train,batch_size=32,epochs=35,validation_data=(X_test,y_test))

y=model3.evaluate(X_test,y_test)

test=pd.read_csv("gdrive/My Drive/test_WyRytb0.csv")

#function to load all images and their corresponding labels
IMAGES_PATH="train/{}"
def load_images_test(data,images_path):
    resize=(150,150)

    x=[]
    
    ids=[]
    i=0
    for img_id in data.values:
        i=i+1
        if(i%1000==0):
           print("{}/{}".format(i,len(data)))
        image=cv2.imread(images_path.format(img_id[0]))
        x.append(cv2.resize(image,resize))
       
        ids.append(img_id[0])
        
    return x,ids

X_sub=load_images_test(test,IMAGES_PATH)

X_sub,names=X_sub

X_sub=np.stack(X_sub,axis=0)

pred=model3.predict_classes(X_sub,batch_size=32)

sub={"image_name":names,"label":pred.tolist()}

dataframe_sub=pd.DataFrame(sub)
dataframe_sub.to_csv("gdrive/My Drive/submissions.csv")

Using TensorFlow backend.
